# Six Degrees of Kevin Bacon

![](images/Kevin_Bacon.jpg)

This activity is motivated by the text **Web Scraping with Python** by Ryan Mitchell, available through O'Reilly [here](http://shop.oreilly.com/product/0636920078067.do).  This book goes in depth with much more on using different libraries with Python around common webscraping tasks and I highly recommend it.  We will focus on the activity of moving from a base page to further pages through their links.  

In [1]:
import requests
from bs4 import BeautifulSoup

### Scraping Links

Below, we take the page dealing with the six degrees of Keving Bacon problem.  Here, our goal is to extract links to other pages that we will subsequently pass to requests.  Recall that a link is located in an `<a>` tag and the link is contained in the `href` attribute.  For example, the tag

```HTML
<a href="/wiki/Six_degrees_of_separation" title="Six degrees of separation">six degrees of separation</a>
```

references the Six Degrees of Separation article.  Note that this is a url within Wikipedia.  We can isolate these inner Wikipedia references.  To begin, let's inspect the link content.

In [2]:
response = requests.get('https://en.wikipedia.org/wiki/Six_Degrees_of_Kevin_Bacon')

In [3]:
soup = BeautifulSoup(response.text, 'html.parser')

In [4]:
for link in soup.find_all('a')[:10]:
    if 'href' in link.attrs:
        print(link.attrs['href'])

#mw-head
#p-search
/wiki/File:Kevin_Bacon.jpg
/wiki/File:Kevin_Bacon.jpg
/wiki/Parlor_game
/wiki/Six_degrees_of_separation
/wiki/Kevin_Bacon
/wiki/Hollywood
/wiki/Kevin_Bacon


Okay, seems there are links outside of the inner wiki links.  However, we see that the wiki links contain `/wiki/`, no colons, and the links are all within the body of the page.  Exploiting these means we can write a regular expression 

```
^(/wiki/)((?!:).)*$
```

that will match only the wiki links.  

In [5]:
import re

In [6]:
for link in soup.find('div', {'id': 'bodyContent'}).find_all('a', href = re.compile('^(/wiki/)((?!:).)*$'))[:10]:
    if 'href' in link.attrs:
        print(link.attrs['href'])

/wiki/Parlor_game
/wiki/Six_degrees_of_separation
/wiki/Kevin_Bacon
/wiki/Hollywood
/wiki/Kevin_Bacon
/wiki/Kevin_Bacon
/wiki/Charitable_organization
/wiki/SixDegrees.org
/wiki/Premiere_(magazine)
/wiki/Kevin_Bacon


### A Function for Links

Now, let's write a function that extracts the link from any wikipedia page.  We should be able to use the idea that the links we care about are located in the same place as our Six Degrees example.  

In [7]:
def get_wikilinks(url):
    links = []
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    for link in soup.find('div', {'id': 'bodyContent'}).find_all('a', href = re.compile('^(/wiki/)((?!:).)*$')):
        links.append(link)
    return links

In [8]:
links = get_wikilinks('https://en.wikipedia.org/wiki/Kevin_Bacon')

In [9]:
links[:10]

[<a class="mw-disambig" href="/wiki/Kevin_Bacon_(disambiguation)" title="Kevin Bacon (disambiguation)">Kevin Bacon (disambiguation)</a>,
 <a href="/wiki/San_Diego_Comic-Con" title="San Diego Comic-Con">San Diego Comic-Con</a>,
 <a href="/wiki/Philadelphia" title="Philadelphia">Philadelphia</a>,
 <a href="/wiki/Pennsylvania" title="Pennsylvania">Pennsylvania</a>,
 <a href="/wiki/Kyra_Sedgwick" title="Kyra Sedgwick">Kyra Sedgwick</a>,
 <a href="/wiki/Sosie_Bacon" title="Sosie Bacon">Sosie Bacon</a>,
 <a href="/wiki/Edmund_Bacon_(architect)" title="Edmund Bacon (architect)">Edmund Bacon</a>,
 <a href="/wiki/Michael_Bacon_(musician)" title="Michael Bacon (musician)">Michael Bacon</a>,
 <a href="/wiki/Footloose_(1984_film)" title="Footloose (1984 film)">Footloose</a>,
 <a href="/wiki/JFK_(film)" title="JFK (film)">JFK</a>]

### Connecting Pages

Now, we want to follow these references, gather more urls, and repeat. For the sake of not running to exhaustion, I abbreviate the output using only a large length requirement for the link list.  To traverse all the pages we would simply change the 

```python
while len(links) > 100:
```

to 

```python
while len(links) > 0:
```

In [10]:
import random
def get_wikilinks(url):
    links = []
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    for link in soup.find('div', {'id': 'bodyContent'}).find_all('a', href = re.compile('^(/wiki/)((?!:).)*$')):
        links.append(link)
    return links

In [13]:
links = get_wikilinks('https://en.wikipedia.org/wiki/Kevin_Bacon')
while len(links) > 100:
    newArticle = 'https://en.wikipedia.org' + links[random.randint(0, len(links)-1)].attrs['href']
    print(newArticle)
    links = get_wikilinks(newArticle)

https://en.wikipedia.org/wiki/Stacy_Keach
https://en.wikipedia.org/wiki/Old_Yeller-Belly
https://en.wikipedia.org/wiki/The_Simpsons_(season_29)
https://en.wikipedia.org/wiki/D%27oh!
https://en.wikipedia.org/wiki/TV_Land
https://en.wikipedia.org/wiki/Nicktoons_(United_States)
https://en.wikipedia.org/wiki/Hallmark_Channel
https://en.wikipedia.org/wiki/Super_Bowl
https://en.wikipedia.org/wiki/New_Haven,_Connecticut
https://en.wikipedia.org/wiki/List_of_Connecticut_locations_by_per_capita_income
https://en.wikipedia.org/wiki/Vernon,_Connecticut
https://en.wikipedia.org/wiki/Pacific_Islander_(U.S._Census)
https://en.wikipedia.org/wiki/Hispanic_and_Latino_Americans
https://en.wikipedia.org/wiki/Chicano
https://en.wikipedia.org/wiki/List_of_U.S._communities_with_Hispanic_majority_populations_in_the_2010_census
https://en.wikipedia.org/wiki/Kendale_Lakes,_Florida
https://en.wikipedia.org/wiki/Native_Alaskan
https://en.wikipedia.org/wiki/2010_United_States_Census
https://en.wikipedia.org/wiki/

### Problem

Write a function to retrieve a list of albums of any area you are interested in using Wikipedia's list of list of albums page: https://en.wikipedia.org/wiki/Lists_of_albums.